# Bedrock with LangChain using a Prompt that includes Context

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Setup

```python
%pip install --upgrade langchain
%pip install -U langchain-aws
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys
import boto3
import botocore
import ipywidgets as widgets
from langchain_aws import BedrockLLM
from langchain.prompts import PromptTemplate
from langchain_aws import ChatBedrock

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, model_formatter, tools

In [3]:
bedrock_runtime_client, bedrock_control_client = bedrock.get_bedrock_clients()

Create new client
  Using region: None
  Using profile: genaidev
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)
Create new client
  Using region: None
  Using profile: genaidev
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Create a reference the Bedrock LLM

In [4]:
dropdown = tools.model_selection(bedrock_control_client)
display(dropdown)

Dropdown(description='Select LLM:', layout=Layout(width='300px'), options={'Claude': 'anthropic.claude-v2:1', …

In [6]:
inference_modifier = {'max_tokens_to_sample': 4096,
                      "temperature": 0.5,
                      "top_k": 250,
                      "top_p": 1,
                      "stop_sequences": ["\n\nHuman"]}

llm = BedrockLLM(model_id=dropdown.value,
                    client=bedrock_runtime_client,
                    model_kwargs=inference_modifier)

## Create a LangChain custom prompt template

In [7]:
# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""
            Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
            <customer_feedback>
            {feedbackFromCustomer}
            </customer_feedback>
            Assistant:"""
            )

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


### Print the number of tokens in the prompt (for interest/debug)

In [8]:
num_tokens = llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

Our prompt has 124 tokens


## Invoke the model and print the response

In [9]:
print(llm.invoke(prompt))

 Here is a draft apology email from Bob to John Doe:

Subject: Apology for the poor service experience 

Dear John,

I am writing to sincerely apologize for the poor service you recently received when contacting our customer support team. Based on the feedback you provided, it is clear we did not meet your expectations for a timely response and resolution. 

I want to apologize that it took 3 days to receive a callback after your initial request. This is not the level of service we aim to provide our valued customers. Additionally, I deeply regret our agent's initial suggested solution was incorrect. It was unacceptable that the issue ultimately took 3 days to resolve. 

We strive to respond to all customer inquiries promptly and fix any issues correctly the first time. Unfortunately, we clearly missed the mark in your case. Please accept my personal apology for the frustration and inconvenience this has caused you and your business.  

I am following up with our support team to review

### Let's try it using Claude v3

In [10]:
dropdown = tools.model_selection(bedrock_control_client)
display(dropdown)

Dropdown(description='Select LLM:', layout=Layout(width='300px'), options={'Claude': 'anthropic.claude-v2:1', …

In [11]:
inference_modifier = {
    "temperature": 0.99,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": []
}

In [12]:
llm = ChatBedrock(model_id=dropdown.value,
            client=bedrock_runtime_client,
            model_kwargs=inference_modifier,
            streaming=True,
)

In [13]:
messages = [
    {"role": "system", "content": "You are a customer service manager."},
    {"role": "human", "content": "Create an apology email from the Service Manager Bob to John Doe in response to the following feedback that was received from the customer:\n\nHello Bob,\nI am very disappointed with the recent experience I had when I called your customer support.\nI was expecting an immediate call back but it took three days for us to get a call back.\nThe first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.\nWe are very unhappy with the response provided and may consider taking our business elsewhere."}
]

In [14]:
response = llm.invoke(messages)

In [15]:
print(response.content)

Dear John Doe,

Please accept my sincere apology for the unsatisfactory experience you had with our customer support. I understand how frustrating it must have been to wait three days for a call back and to receive an incorrect initial solution.

As the Service Manager, I take full responsibility for this incident and the inconvenience it has caused you. We pride ourselves on providing prompt and effective support, and clearly, we fell short in this case. 

I have thoroughly reviewed the details of your interaction with our team, and I can assure you that we have taken immediate steps to address the issues that led to this poor customer experience. We have retrained our support staff on proper escalation procedures and have implemented additional quality checks to ensure that similar situations do not occur in the future.

Please know that your satisfaction is of the utmost importance to us. I would like to offer you a 20% discount on your next purchase as a gesture of goodwill and to 